# Graphs and Functions


## Setup

Import libraries:

In [1]:
import tensorflow as tf

2022-10-30 15:16:49.249646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-30 15:16:49.315065: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-30 15:16:49.331395: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-30 15:16:49.629901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
tf.config.list_physical_devices()

2022-10-30 15:18:03.387106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-30 15:18:03.415514: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-30 15:18:03.415584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Taking Advantage of Graphs

`tf.function` takes a regular function as input and returns a `Function` object. The result `Function` object is a callable that will build graph by intracpecting python function (how?) and will have the same contract as the wrapped original function.  

### Basic Example

In [3]:
@tf.function
def linear_operator(w, b, x):
    return tf.matmul(w, x) + b

linear_operator

In [9]:
w=tf.constant([[1.0, 2.0], [3.0, 4.0]]),
b=tf.constant([[1.0], [2.0]]),
x=tf.Variable([[3.0], [4.0]])

linear_operator(w, b, x)

<tf.Tensor: shape=(1, 2, 1), dtype=float32, numpy=
array([[[12.],
        [27.]]], dtype=float32)>

In [11]:
tf.matmul(w, x) + b == linear_operator(w, b, x)

<tf.Tensor: shape=(1, 2, 1), dtype=bool, numpy=
array([[[ True],
        [ True]]])>

### Explanation

`Function` encapsulates multiple `tf.Graph` behind the same API. It creates a new Graph depending on the `dtype` and `shape` of the arguments.

### Inner Functions

`tf.function` applies to the function and all the function it calls:

In [12]:
def inner_function(w, b, x):
    print("From inner function!")
    return tf.matmul(w, x) + b

@tf.function
def outer_function(w, b, x, z):
    print("From outer function!")
    return tf.reduce_sum(inner_function(w, b, x) * z)

In [15]:
z = tf.constant([[2.0], [1.0]])

outer_function(w, b, x, z)

From outer function!
From inner function!


<tf.Tensor: shape=(), dtype=float32, numpy=51.0>

In [16]:
outer_function(w, b, x, z)

<tf.Tensor: shape=(), dtype=float32, numpy=51.0>

**NOTE**: The second time we see not python side effects!

## Converting Python Functions to Graphs

